In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: 4a5e0119-fb85-40fd-87ab-f201a2aa0fde: GSB - UHE-MACHADINHO>, <Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: b75339ff-e91f-4e75-a082-5d7e23a4d049: GSB - UHE-SALTO SANTIAGO>, <Company: ed749424-27e4-4342-82cd-f924de71830b: Limbo - UTPS>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA>, <Company: 7fe7b473-6d9c-49d6-a9ca-74f3424d3211: GSB - UHE-SALTO OSÓRIO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: d25d0eb2-61a8-4646-8e0e-31c8e68cf216: TESTE>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 1b

In [3]:
import numpy
from numpy import arange
from helpers.km_converter import km_to_coordinates
from collections import defaultdict
from openpyxl import Workbook
from django.db.models import Q
from tqdm.notebook import tqdm
from datetime import datetime
import pytz
from apps.service_orders.const import resource_approval_status

In [4]:
companies_name = list(Company.objects.filter(name__startswith='GES').values_list('name',flat=True))
companies_name

['GES - UHE-CANA BRAVA',
 'GES - UHE-ESTREITO',
 'GES - PCH-RONDONÓPOLIS',
 'GES - UHE-MACHADINHO',
 'GES - UHE-SALTO SANTIAGO',
 'GES - PCH-JOSÉ GELÁZIO DA ROCHA',
 'GES - UHE-SÃO SALVADOR',
 'GES - GEH',
 'GES - UHE-ITA',
 'GES - UHE-PASSO FUNDO',
 'GES - UHE-SALTO OSORIO',
 'GES - UHE-JAGUARA',
 'GES - UHE-MIRANDA',
 'GES - UHE-PONTE DE PEDRA']

In [5]:
def get_permission(obj, company):
    try:            
        user_in_company = next(
            a
            for a in obj.companies_membership.all()
            if a.company == company
        )
        return user_in_company.permissions.name
    except Exception:
        return None

In [6]:
date_to_use = datetime(2023,9,1).replace(tzinfo=pytz.timezone("America/Sao_Paulo"))
date_to_use2 = datetime(2023,9,30, 23, 59).replace(tzinfo=pytz.timezone("America/Sao_Paulo"))

date_to_use2

datetime.datetime(2023, 9, 30, 23, 59, tzinfo=<DstTzInfo 'America/Sao_Paulo' LMT-1 day, 20:54:00 STD>)

In [7]:
companies = Company.objects.filter(name__in=companies_name).order_by('name')

In [8]:
for company in companies:
    print("\n")
    print(company.name)
    print("\n")
#     reportings = Reporting.objects.filter(company=company, updated_at__gte=date_to_use)

    engagemment_objects = ['OccurrenceRecord', 'OperationalControlRecord', 'Procedure', 'ServiceOrder', 'ServiceOrderResource', 'ServiceOrderAction']
    user_exclude = ['engie']


    user_moves = []
    for user in UserInCompany.objects.filter(company=company).exclude(user__username__in=user_exclude).exclude(permissions__name__icontains='inativ'):
        num_moves = 0
        for item in engagemment_objects:
            if item != 'ServiceOrder' and item != 'ServiceOrderResource':
                num_moves += eval(item).history.filter(history_date__gte=date_to_use, history_date__lte=date_to_use2, history_user=user.user, firm__company=company).count()
            elif item == 'ServiceOrderResource':
                num_moves += eval(item).history.filter(history_date__gte=date_to_use, history_date__lte=date_to_use2, history_user=user.user, entity__company=company).count()
            elif item == 'ServiceOrder':
                num_moves += eval(item).history.filter(history_date__gte=date_to_use, history_date__lte=date_to_use2, history_user=user.user, company=company).count()
#         for item in engagemment_objects:
#             if item != 'ServiceOrder' and item != 'ServiceOrderResource':
#                 num_moves += eval(item).history.filter(history_user=user, firm__company=company).count()
#             elif item == 'ServiceOrderResource':
#                 num_moves += eval(item).history.filter(history_user=user, entity__company=company).count()
#             elif item == 'ServiceOrder':
#                 num_moves += eval(item).history.filter(history_user=user, company=company).count()
        user_moves.append((num_moves, user))

    user_moves.sort(key=lambda tup: tup[0], reverse=True)
    print(f'Usuário | Nível de acesso | Engajamento')
    for user in user_moves:
        print(f'{user[1].user.get_full_name()} | {get_permission(user[1].user, company).capitalize()} | {user[0]}')
#     active_users = len([user for user in user_moves if user[0]>0])
#     print(f'Quantidade de usuários: {len(user_moves)} / Porcentagem de usuários ativos: {(active_users/len(user_moves))*100:.2f}%')



GES - GEH


Usuário | Nível de acesso | Engajamento
Gilnei Minella | Ges - homologador | 35
Alice Neves Fernandes | Ges - líder técnico e comercial externo | 29
Rui Pinali Biavatti | Ges - homologador | 0
Welviston Marques Silva | Ges - homologador | 0
Marcelo Brugnaro Schultz | Ges - homologador | 0
Victor Martins | Ges - administrador | 0
Michel Luiz Rodovanski | Ges - homologador | 0
Karla Zorzam | Ges - administrador | 0
Jamilto Ferreira Cardoso | Ges - homologador | 0
Evandro Ferreira | Ges - administrador | 0
Diego Collet | Ges - gerencial | 0
Camila de Carvalho Demétrio Manoel | Ges - líder técnico e comercial externo de monitoramentos | 0
Luiz Carlos Locks Junior | Ges - lider comercial interno | 0
Alex Correia | Ges - homologador | 0
Rogério Suematsu | Ges - homologador | 0
Alejandra Dalotto | Ges - líder técnico e comercial externo | 0
Josuel Hartmam | Ges - lider comercial interno | 0
Antônio C. Martins | Ges - gerencial | 0
Paulo Henrique Santos Feitosa | Ges - homologado

Carlos  Alberto Sousa Silva | Ges - lider técnico interno | 0


GES - UHE-CANA BRAVA


Usuário | Nível de acesso | Engajamento
IHM UHCB01 | Ges - homologador | 30
Flavio Junior Pires De Oliveira | Ges - lider técnico interno | 14
Rodrigo Rafael Sanjuan | Ges - líder técnico e comercial externo | 9
Dhonata Luiz Teixeira | Ges - lider técnico externo | 5
Simone Rodrigues Gonçalves | Ges - homologador | 3
Cid Ionceck | Ges - homologador | 3
Edijan Corrêa | Ges - líder técnico e comercial externo | 1
Welviston Marques Silva | Ges - gerencial | 0
Victor Martins | Ges - administrador | 0
Rivelino Augusto Dos Reis | Ges - lider técnico interno | 0
Ilmar Goltara Gomes | Ges - analista de processos | 0
Cleuton Pacheco | Ges - colaborador interno | 0
Rafaele Costa Delboni | Ges - lider técnico interno | 0
Karla Zorzam | Ges - administrador | 0
José Carlos Borgmann | Ges - lider técnico interno | 0
Bianca Savana Sell | Ges - lider técnico interno | 0
Laura de Mello Holme | Ges - lider técnico int

Wilherman Caitano Do Rego | Ges - colaborador interno | 0
Hilquias dos Santos Silva | Ges - homologador | 0
André Gonçalves Ferreira | Ges - colaborador externo | 0
Antônio Mailson Monteiro De Freitas | Ges - colaborador interno | 0
Wellinghton Aguiar Dos Santos | Ges - colaborador interno | 0
Higgor Batista Moura | Ges - lider técnico interno | 0
Integração Forms SST | Ges - integrações | 0
Edi João Stibuski | Ges - lider técnico externo | 0
Leandro Vieira Santos Kroger | Ges - homologador | 0
Alice Neves Fernandes | Ges - líder técnico e comercial externo | 0
IHM UHET01 | Ges - homologador | 0
Lauri Alves da Rocha Junior | Ges - lider técnico interno | 0
Grasiela Fagundes Minatto Cardoso | Ges - analista de processos | 0
Marcos Roberto Castanheira | Ges - gerencial | 0
Joseany Meireles Cantanhede | Ges - lider técnico interno | 0
Beusilay Alves Coelho | Ges - lider técnico interno | 0
Maria Nazare Munari Angeloni Hahne | Ges - administrador | 0
Jose Vicente Miranda Rescigno | Ges - l

Ana Alt | Ges - líder técnico e comercial externo | 0
Alice Neves Fernandes | Ges - líder técnico e comercial externo | 0
Jane Agnol | Ges - lider comercial interno | 0
Milena Hardt | Ges - colaborador externo | 0
Sérgio Luiz Souza | Ges - homologador | 0
Renato Regoso | Ges - lider técnico interno | 0
Carlos Andre Segatto | Ges - lider técnico interno | 0
Bruno Luan Kuhn | Ges - lider técnico interno | 0
Josemar Lorscheiter | Ges - lider técnico externo | 0
Rodrigo Daltoé | Ges - lider técnico interno | 0
Grasiela Fagundes Minatto Cardoso | Ges - analista de processos | 0
Maria Nazare Munari Angeloni Hahne | Ges - administrador | 0
Sibeli Juliana Mello Menegatti | Ges - lider comercial interno | 0
Jose Vicente Miranda Rescigno | Ges - lider técnico interno | 0
Júlio Cesar Pereira | Ges - lider técnico interno | 0
Camila Costa De Oliveira | Ges - lider técnico interno | 0
Joao Carlos Truylio | Ges - colaborador interno | 0
Rafael Vendruscolo | Ges - lider comercial externo | 0
Artur Ro

Rosangela Korbes Burg | Ges - lider técnico interno | 0
Andréia Zanatta | Ges - administrador | 0
Giulia Platt Maffezzolli | Ges - administrador | 0
Eduardo Marchante | Ges - líder técnico e comercial externo | 0
Rafael Silva De Almeida | Ges - lider técnico interno | 0
Edijan Corrêa | Ges - líder técnico e comercial externo | 0
Gilmar Korb | Ges - lider comercial interno | 0


GES - UHE-MACHADINHO


Usuário | Nível de acesso | Engajamento
Rosenei Olicheski | Ges - colaborador externo | 75
Tailan da Silva | Ges - lider técnico externo | 60
Scheila Pimentel | Ges - homologador | 30
Gilnei Minella | Ges - homologador | 24
Ericsson Miquett Oliveira | Ges - homologador | 9
Clademir Mandelli | Ges - lider técnico interno | 8
Juliana Henzel | Ges - líder técnico e comercial externo | 7
Rodrigo Rafael Sanjuan | Ges - líder técnico e comercial externo | 6
Fernanda Franciele Brol | Ges - líder técnico e comercial externo de monitoramentos | 4
Jordana Maroso | Ges - líder técnico e comercial ext

Evandro Ferreira | Ges - administrador | 0
José Lourival Magri | Ges - gerencial | 0
Luis Ricardo Dos Reis Caldeira | Ges - colaborador interno | 0
Fabricio Carvalho De Paula | Ges - lider técnico interno | 0
EXITON FERREIRA | Ges - lider técnico externo | 0
Marco Antonio Barreto Jarces | Ges - lider comercial interno | 0
Thais Mariano Machado Alkmim | Ges - lider técnico interno | 0
Carlos Cesar Schmitt da Luz | Ges - lider técnico interno | 0
Ana Luisa Moreira Ferreira | Ges - líder técnico e comercial externo | 0
Kenifer Nascimento Ferreira | Ges - lider comercial interno | 0
Luis Guilherme De Oliveira Miranda | Ges - analista de processos | 0
Luiz Carlos Locks Junior | Ges - lider comercial interno | 0
Lucas Machado de Melo Reis | Ges - líder técnico e comercial externo | 0
Allan Keller | Ges - administrador | 0
Rogério Suematsu | Ges - gerencial | 0
Luanda Santos | Ges - lider técnico interno | 0
Anaisa Silva Felice | Ges - colaborador externo | 0
Adriano de Castro Cerveira Gomes 

Carlos Andre Segatto | Ges - lider técnico interno | 0
Bruno Luan Kuhn | Ges - lider técnico interno | 0
Rodrigo Daltoé | Ges - lider técnico interno | 0
Grasiela Fagundes Minatto Cardoso | Ges - analista de processos | 0
Olmir Atilio Brustolin | Ges - colaborador interno | 0
Maria Nazare Munari Angeloni Hahne | Ges - administrador | 0
Sibeli Juliana Mello Menegatti | Ges - lider comercial interno | 0
Jose Vicente Miranda Rescigno | Ges - lider técnico interno | 0
Júlio Cesar Pereira | Ges - lider técnico interno | 0
Camila Costa De Oliveira | Ges - lider técnico interno | 0
Artur Roberto Frota Ellwanger | Ges - gerencial | 0
Camila Mendes | Ges - lider comercial interno | 0
Pedro Flávio Portela | Ges - líder técnico e comercial externo | 0
Tulio Ribeiral | Ges - administrador | 0
Fernando Gritti | Ges - lider técnico interno | 0
Chandrieli Simon | Ges - lider técnico externo | 0
Giovanna Schmidt Luz | Ges - lider técnico interno | 0
Rafael Dias Portela Oliveira | Ges - lider técnico e

Victor Martins | Ges - administrador | 0
Rivelino Augusto Dos Reis | Ges - lider técnico interno | 0
Ilmar Goltara Gomes | Ges - analista de processos | 0
Cleuton Pacheco | Ges - colaborador interno | 0
Felipe Costa | Ges - homologador | 0
Rafaele Costa Delboni | Ges - lider técnico interno | 0
Karla Zorzam | Ges - administrador | 0
Ivair Antonio Carraro | Ges - lider técnico interno | 0
Rober Joni Sossella | Ges - lider técnico interno | 0
José Carlos Borgmann | Ges - lider técnico interno | 0
Bianca Savana Sell | Ges - lider técnico interno | 0
Jamilto Ferreira Cardoso | Ges - colaborador externo | 0
Alysson José Schawalb | Ges - colaborador externo | 0
Evandro Ferreira | Ges - administrador | 0
Clovis Tosin da Silva | Ges - homologador | 0
Daniel de Freitas Balbi | Ges - líder técnico e comercial externo | 0
José Lourival Magri | Ges - gerencial | 0
Cleito Mazzuco | Ges - lider técnico interno | 0
Jose Clovis Do Vale Junior | Ges - lider técnico interno | 0
Carlos Cesar Schmitt da L

Luanda Santos | Ges - lider técnico interno | 0
Alejandra Dalotto | Ges - líder técnico e comercial externo | 0
Carlos Henrique Orsi | Ges - colaborador externo | 0
Pedro Rogério Leandro Da Silva | Ges - colaborador externo | 0
Ricardo Muller Arcari | Ges - líder técnico e comercial externo | 0
Gilberto Perico | Ges - colaborador interno | 0
Antônio C. Martins | Ges - gerencial | 0
José Carlos B. Junior | Ges - colaborador interno | 0
Carli Ines B. Giacomini | Ges - lider comercial interno | 0
Lucas Tarso Neier Riccio | Ges - lider técnico interno | 0
Nilton C. Da Silva França | Ges - lider técnico interno | 0
Ediverte Amorim | Ges - lider técnico interno | 0
Celito Correa Junior | Ges - lider técnico interno | 0
Douglas N. Lirio Ribeiro | Ges - lider técnico interno | 0
Felipe Blanger | Ges - colaborador interno | 0
Davi Falegue Mantaro Menghi Nishimura | Ges - administrador | 0
Camila Rodriguez | Ges - analista de processos | 0
Clayton Ferreira Rodrigues Dos Reis | Ges - colaborador 

Diogo Marcello Cardoso Modro | Ges - lider técnico interno | 0
Igor Renan Braga Dos Santos | Ges - homologador | 0
Rosangela Korbes Burg | Ges - lider técnico interno | 0
Andréia Zanatta | Ges - administrador | 0
Giulia Platt Maffezzolli | Ges - administrador | 0
Eduardo Marchante | Ges - colaborador interno | 0
Rafael Silva De Almeida | Ges - lider técnico interno | 0
Willyam Ribeiro Marques | Ges - colaborador interno | 0


In [9]:
# for company in companies:
    
#     print("\n")
#     print(company.name)
#     print("\n")
#     print(f'Usuário\tEmail')
    
#     user_exclude = ['engie']
    
#     for user in UserInCompany.objects.filter(company=company).exclude(user__username__in=user_exclude).order_by('permissions__name', 'user__first_name'):
# #         permission=get_permission(user, company)
# #         if permission.capitalize() == 'Homologador':
#         print(f'{user.user.get_full_name()}\t{user.permissions.name}\t{user.user.email}')
# #         else:
# #             pass

In [10]:
def get_total_price(obj):
    total_price = 0
    for resource in obj.resources.all():
        try:
            resource_total_price = resource.unit_price * resource.amount
            if not isinstance(resource_total_price, (int, float)):
                raise Exception()
            total_price += resource_total_price
        except Exception:
            continue

    return total_price

In [11]:
def get_spent_price(obj):

    spent_price = 0
    for resource in obj.resources.all():
        for procedure_resource in resource.serviceorderresource_procedures.all():
            try:
                resource_spent_price = ((procedure_resource.unit_price * procedure_resource.amount) if procedure_resource.approval_status == resource_approval_status.APPROVED_APPROVAL else 0)
                if not isinstance(resource_spent_price, (int, float)):
                    raise Exception()
                spent_price += resource_spent_price
            except Exception:
                continue

    return spent_price

In [21]:
months_name={'9': 'Setembro'}

In [22]:
print('Número de recursos consumidos (%)')

contracts=[]

for company in companies:
#     for i in range(1, len(months_name)+1):
    contracts=Contract.objects.filter(firm__company=company, status__name__icontains='em vigência')
    total_price=0
    spent_price=0
    for contract in contracts:
        spent_price+=get_spent_price(contract)
        total_price+=get_total_price(contract)
    try:
        print(f'{company.name} | {round(spent_price/total_price * 100, 2)}%')
    except ZeroDivisionError:
        print(f'{company.name} | 0.0%')

Número de recursos consumidos (%)
GES - GEH | 31.57%
GES - PCH-JOSÉ GELÁZIO DA ROCHA | 0.0%
GES - PCH-RONDONÓPOLIS | 0.0%
GES - UHE-CANA BRAVA | 1.13%
GES - UHE-ESTREITO | 4.49%
GES - UHE-ITA | 66.62%
GES - UHE-JAGUARA | 49.12%
GES - UHE-MACHADINHO | 60.54%
GES - UHE-MIRANDA | 32.89%
GES - UHE-PASSO FUNDO | 59.56%
GES - UHE-PONTE DE PEDRA | 0.0%
GES - UHE-SALTO OSORIO | 60.0%
GES - UHE-SALTO SANTIAGO | 60.0%
GES - UHE-SÃO SALVADOR | 0.0%


In [23]:
print('Vinculação de serviços')

for company in companies:
    for k, v in months_name.items():
        occurrences_total=OccurrenceRecord.objects.filter(company=company, created_at__year=2023, created_at__month=int(k), operational_control__isnull=True).count()
        occurrences_services=OccurrenceRecord.objects.filter(company=company, created_at__year=2023, created_at__month=int(k), operational_control__isnull=True, service_orders__isnull=False).count()
        try:
            print(f'{company.name} - {v} | {round(occurrences_services/occurrences_total * 100, 2)}%')
        except ZeroDivisionError:
            print(f'{company.name} - {v} | 0.0%')

Vinculação de serviços
GES - GEH - Setembro | 0.0%
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 0.0%
GES - PCH-RONDONÓPOLIS - Setembro | 0.0%
GES - UHE-CANA BRAVA - Setembro | 0.0%
GES - UHE-ESTREITO - Setembro | 0.0%
GES - UHE-ITA - Setembro | 3.42%
GES - UHE-JAGUARA - Setembro | 0.0%
GES - UHE-MACHADINHO - Setembro | 55.56%
GES - UHE-MIRANDA - Setembro | 0.0%
GES - UHE-PASSO FUNDO - Setembro | 0.0%
GES - UHE-PONTE DE PEDRA - Setembro | 0.0%
GES - UHE-SALTO OSORIO - Setembro | 0.0%
GES - UHE-SALTO SANTIAGO - Setembro | 0.0%
GES - UHE-SÃO SALVADOR - Setembro | 0.0%


In [24]:
print('Emissão de boletim de medição')

for company in companies:
    for k, v in months_name.items():
        bulletin_count=MeasurementBulletin.objects.filter(Q(contract__firm__company=company)|Q(contract__subcompany__company=company), creation_date__year=2023, creation_date__month=int(k)).count()
        print(f'{company.name} - {v} | {bulletin_count}')

Emissão de boletim de medição
GES - GEH - Setembro | 3
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 1
GES - PCH-RONDONÓPOLIS - Setembro | 1
GES - UHE-CANA BRAVA - Setembro | 1
GES - UHE-ESTREITO - Setembro | 1
GES - UHE-ITA - Setembro | 5
GES - UHE-JAGUARA - Setembro | 5
GES - UHE-MACHADINHO - Setembro | 2
GES - UHE-MIRANDA - Setembro | 2
GES - UHE-PASSO FUNDO - Setembro | 3
GES - UHE-PONTE DE PEDRA - Setembro | 1
GES - UHE-SALTO OSORIO - Setembro | 1
GES - UHE-SALTO SANTIAGO - Setembro | 1
GES - UHE-SÃO SALVADOR - Setembro | 1


In [25]:
print('Número de rg homologados')

for company in companies:
    for k, v in months_name.items():
        homologado_count=len(list(set(OccurrenceRecord.history.model.objects.filter(company=company, history_date__year=2023, history_date__month=k, approval_step__name__icontains='deferido').values_list('uuid'))))
        print(f'{company.name} - {v} | {homologado_count}')

Número de rg homologados
GES - GEH - Setembro | 0
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 0
GES - PCH-RONDONÓPOLIS - Setembro | 0
GES - UHE-CANA BRAVA - Setembro | 1
GES - UHE-ESTREITO - Setembro | 0
GES - UHE-ITA - Setembro | 6
GES - UHE-JAGUARA - Setembro | 2
GES - UHE-MACHADINHO - Setembro | 18
GES - UHE-MIRANDA - Setembro | 1
GES - UHE-PASSO FUNDO - Setembro | 1
GES - UHE-PONTE DE PEDRA - Setembro | 0
GES - UHE-SALTO OSORIO - Setembro | 0
GES - UHE-SALTO SANTIAGO - Setembro | 0
GES - UHE-SÃO SALVADOR - Setembro | 0


In [26]:
print('Número de registros operacionais')

for company in companies:
    for k, v in months_name.items():
        occurrences_operational=OccurrenceRecord.objects.filter(company=company, created_at__year=2023, created_at__month=int(k), operational_control__isnull=False).count()
        print(f'{company.name} - {v} | {occurrences_operational}')

Número de registros operacionais
GES - GEH - Setembro | 0
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 0
GES - PCH-RONDONÓPOLIS - Setembro | 0
GES - UHE-CANA BRAVA - Setembro | 15
GES - UHE-ESTREITO - Setembro | 3322
GES - UHE-ITA - Setembro | 341
GES - UHE-JAGUARA - Setembro | 0
GES - UHE-MACHADINHO - Setembro | 0
GES - UHE-MIRANDA - Setembro | 0
GES - UHE-PASSO FUNDO - Setembro | 0
GES - UHE-PONTE DE PEDRA - Setembro | 0
GES - UHE-SALTO OSORIO - Setembro | 81
GES - UHE-SALTO SANTIAGO - Setembro | 51
GES - UHE-SÃO SALVADOR - Setembro | 0


In [27]:
print('Número de rg ambiental')

for company in companies:
    for k, v in months_name.items():
        occurrences_ambiental=OccurrenceRecord.objects.filter(company=company, created_at__year=2023, created_at__month=int(k), operational_control__isnull=True, search_tags__name='Ambiental').count()
        print(f'{company.name} - {v} | {occurrences_ambiental}')

Número de rg ambiental
GES - GEH - Setembro | 0
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 0
GES - PCH-RONDONÓPOLIS - Setembro | 0
GES - UHE-CANA BRAVA - Setembro | 0
GES - UHE-ESTREITO - Setembro | 0
GES - UHE-ITA - Setembro | 6
GES - UHE-JAGUARA - Setembro | 0
GES - UHE-MACHADINHO - Setembro | 15
GES - UHE-MIRANDA - Setembro | 0
GES - UHE-PASSO FUNDO - Setembro | 2
GES - UHE-PONTE DE PEDRA - Setembro | 0
GES - UHE-SALTO OSORIO - Setembro | 8
GES - UHE-SALTO SANTIAGO - Setembro | 8
GES - UHE-SÃO SALVADOR - Setembro | 1


In [28]:
print('Número de rg patrimonial')

for company in companies:
    for k, v in months_name.items():
        occurrences_ambiental=OccurrenceRecord.objects.filter(company=company, created_at__year=2023, created_at__month=int(k), operational_control__isnull=True, search_tags__name='Patrimonial').count()
        print(f'{company.name} - {v} | {occurrences_ambiental}')

Número de rg patrimonial
GES - GEH - Setembro | 0
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 0
GES - PCH-RONDONÓPOLIS - Setembro | 0
GES - UHE-CANA BRAVA - Setembro | 1
GES - UHE-ESTREITO - Setembro | 0
GES - UHE-ITA - Setembro | 140
GES - UHE-JAGUARA - Setembro | 5
GES - UHE-MACHADINHO - Setembro | 3
GES - UHE-MIRANDA - Setembro | 0
GES - UHE-PASSO FUNDO - Setembro | 4
GES - UHE-PONTE DE PEDRA - Setembro | 0
GES - UHE-SALTO OSORIO - Setembro | 4
GES - UHE-SALTO SANTIAGO - Setembro | 31
GES - UHE-SÃO SALVADOR - Setembro | 1


In [29]:
print('Número de serviços')

for company in companies:
    for k, v in months_name.items():
        services=ServiceOrder.objects.filter(company=company, opened_at__year=2023, opened_at__month=int(k)).count()
        print(f'{company.name} - {v} | {services}')

Número de serviços
GES - GEH - Setembro | 0
GES - PCH-JOSÉ GELÁZIO DA ROCHA - Setembro | 1
GES - PCH-RONDONÓPOLIS - Setembro | 1
GES - UHE-CANA BRAVA - Setembro | 1
GES - UHE-ESTREITO - Setembro | 1
GES - UHE-ITA - Setembro | 0
GES - UHE-JAGUARA - Setembro | 3
GES - UHE-MACHADINHO - Setembro | 0
GES - UHE-MIRANDA - Setembro | 1
GES - UHE-PASSO FUNDO - Setembro | 0
GES - UHE-PONTE DE PEDRA - Setembro | 1
GES - UHE-SALTO OSORIO - Setembro | 1
GES - UHE-SALTO SANTIAGO - Setembro | 1
GES - UHE-SÃO SALVADOR - Setembro | 1
